<a href="https://colab.research.google.com/github/groda/big_data/blob/master/generate_data_with_Faker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/groda/big_data"><div><img src="https://github.com/groda/big_data/blob/master/logo_bdb.png?raw=true" align=right width="90"></div></a>

# Need test data?
<br>
<br>


Here's a quick way to generate some fake data using the Python `Faker` library ([https://faker.readthedocs.io/](https://faker.readthedocs.io/)).

**Note:** this is not _synthetic_ data as it is generated with simple methods and will most likely not fit any real-life distribution. Still, it can be useful for test purposes when no data is at hand.

# Install Faker

The Python `faker` module needs to be installed. Note that on Google Colab you can use `!pip` as well as just `pip` (no exclamation mark).

In [1]:
pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.7 MB/s eta 0:00:00


# Generate a Pandas dataframe with fake data

Import `Faker` and set a random seed ($42$).

In [2]:
from faker import Faker
# Set the seed value of the shared `random.Random` object
# across all internal generators that will ever be created
Faker.seed(42)

`fake` is a fake data generator with `DE_de` locale.

In [3]:
fake = Faker('de_DE')
fake.seed_locale('de_DE', 42)
# Creates and seeds a unique `random.Random` object for
# each internal generator of this `Faker` instance
fake.seed_instance(42)

Import pandas to dave data into a dataframe

In [4]:
import pandas as pd

The function `create_row_faker` creates one row of fake data. Here we choose to generate a row containing the following fields:
 - `fake.name()`
 - `fake.postcode()`
 - `fake.email()`
 - `fake.country()`.

In [5]:
def create_row_faker(num=1):
    output = [{"name": fake.name(),
               "age": fake.random_int(0, 100),
               "postcode": fake.postcode(),
               "email": fake.email(),
               "nationality": fake.country(),
              } for x in range(num)]
    return output

Generate a single row

In [6]:
create_row_faker()

[{'name': 'Aleksandr Weinhage',
  'age': 35,
  'postcode': '32181',
  'email': 'bbeckmann@example.org',
  'nationality': 'Fidschi'}]

Generate a dataframe `df_fake` of 5000 rows using `create_row_faker`.

We're using the _cell magic_ `%%time` to time the operation.

In [7]:
%%time
df_fake = pd.DataFrame(create_row_faker(5000))

CPU times: user 615 ms, sys: 8.48 ms, total: 624 ms
Wall time: 709 ms


View dataframe

In [8]:
df_fake

,name,age,postcode,email,nationality
0,Prof. Kurt Bauer B.A.,91,37940,hildaloeffler@example.com,Guatemala
1,Ekkehart Wilms-Kallert,13,61559,maja07@example.net,Brasilien
2,Annelise Röhrdanz-Hornig,80,93103,daniel31@example.com,Guatemala
3,Magrit Knappe B.A.,47,34192,gottliebmitschke@example.com,Guadeloupe
4,Univ.Prof. Gotthilf Wirth B.Sc.,29,56413,heini76@example.net,Litauen
...,...,...,...,...,...
4995,Janusz Nette,74,33091,ann-kathrinsiering@example.net,Belarus
4996,Frau Cathleen Bähr,97,89681,hethurhubertus@example.org,St. Barthélemy
4997,Ulla Seidel,66,28358,klotzbabette@example.net,St. Lucia
4998,Janin Speer MBA.,64,76879,giesskarl-hermann@example.com,Kroatien


For more fake data generators see Faker's [standard providers](https://faker.readthedocs.io/en/master/providers.html#standard-providers) as well as [community providers](https://faker.readthedocs.io/en/master/communityproviders.html#community-providers).

# Generate PySpark dataframe with fake data

Install PySpark.

In [9]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=557696349679c00f57e100d96a889e40e3cb7b25df3f8be22a8c4ad3743ef3ec
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [10]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Faker demo") \
    .getOrCreate()

In [11]:
df = spark.createDataFrame(create_row_faker(5000))

To avoid getting the warning, either use [pyspark.sql.Row](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Row) and let Spark infer datatypes or create a schema for the dataframe specifying the datatypes of all fields (here's the list of all [datatypes](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=types#module-pyspark.sql.types)).

In [12]:
from pyspark.sql.types import *
schema = StructType([StructField('name', StringType()),
                     StructField('age',IntegerType()),
                     StructField('postcode',StringType()),
                     StructField('email', StringType()),
                     StructField('nationality',StringType())])

In [13]:
df = spark.createDataFrame(create_row_faker(5000), schema)

In [14]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- postcode: string (nullable = true)
 |-- email: string (nullable = true)
 |-- nationality: string (nullable = true)



Let's generate some more data (dataframe with $5\cdot10^4$ rows). The file will be partitioned by Spark.

In [15]:
%%time
n = 5*10**4
df = spark.createDataFrame(create_row_faker(n), schema)

CPU times: user 4.75 s, sys: 58.6 ms, total: 4.81 s
Wall time: 6.45 s


It took a long time (~4 sec. for 50000 rows)!

Can we do better?

The function `create_row_faker()` returns a list. This is not efficient, what we need is a _generator_ instead.

In [16]:
d = create_row_faker(5)
# what type is d?
type(d)

list

Now `d` is a generator

In [17]:
d = ({"name": fake.name(),
      "age": fake.random_int(0, 100),
      "postcode": fake.postcode(),
      "email": fake.email(),
      "nationality": fake.country()} for i in range(5))
# what type is d?
type(d)

generator

In [18]:
%%time
n = 5*10**4
d = ({"name": fake.name(),
      "age": fake.random_int(0, 100),
      "postcode": fake.postcode(),
      "email": fake.email(),
      "nationality": fake.country()}
     for i in range(n))
df = spark.createDataFrame(d, schema)

CPU times: user 3.78 s, sys: 28.3 ms, total: 3.8 s
Wall time: 3.92 s


This wasn't faster.

I will look into how one can leverage Hadoop's parallelism to generate dataframes and speed the process.

## Filter and aggregate with PySpark

In [19]:
type(df)

pyspark.sql.dataframe.DataFrame

Show the first five records in the dataframe of fake data.

In [20]:
df.show(n=5, truncate=False)

+---------------------+---+--------+------------------------+-----------+
|name                 |age|postcode|email                   |nationality|
+---------------------+---+--------+------------------------+-----------+
|Rudolf Rust          |76 |86566   |kornelia11@example.org  |Indonesien |
|Kerstin Putz-Köster  |54 |18953   |qrudolph@example.org    |Kenia      |
|Ivo Schinke B.Sc.    |3  |60202   |radischjames@example.net|Chile      |
|Hans-Henning Staude  |82 |68552   |ladislaus89@example.com |Thailand   |
|Justine Weinhage B.A.|53 |62346   |ytaesche@example.org    |Österreich |
+---------------------+---+--------+------------------------+-----------+
only showing top 5 rows



Do some data aggregation:
 - group by postcode
 - count the number of persons and the average age for each postcode
 - filter out postcodes with less than 4 persons
 - sort by average age descending
 - show the first 5 entries

In [21]:
import pyspark.sql.functions as F
df.groupBy('postcode') \
  .agg(F.count('postcode').alias('Count'), F.round(F.avg('age'), 2).alias('Average age')) \
  .filter('Count>3') \
  .orderBy('Average age', ascending=False) \
  .show(5)

+--------+-----+-----------+
|postcode|Count|Average age|
+--------+-----+-----------+
|   18029|    4|      91.75|
|   67611|    4|       87.0|
|   47898|    4|       85.5|
|   45386|    4|      82.75|
|   46755|    4|       78.5|
+--------+-----+-----------+
only showing top 5 rows



Postcode $18029$ has the highest average age ($91.75$). Show all entries for postcode $18029$ using `filter`.

In [22]:
df.filter('postcode==18029').show(truncate=False)

+---------------------------+---+--------+------------------------------+------------------+
|name                       |age|postcode|email                         |nationality       |
+---------------------------+---+--------+------------------------------+------------------+
|Univ.Prof. Roderich Liebelt|89 |18029   |anne-katrinscholtz@example.com|Grönland          |
|Herwig Matthäi B.A.        |90 |18029   |steinberggerta@example.com    |Amerikanisch-Samoa|
|Univ.Prof. Mijo Weihmann   |92 |18029   |hoevelantonius@example.net    |Niederlande       |
|Aynur Karz B.Eng.          |96 |18029   |cschleich@example.com         |Puerto Rico       |
+---------------------------+---+--------+------------------------------+------------------+



# Another example with multiple locales and weights

We are going to use multiple locales with weights (following the [examples](https://faker.readthedocs.io/en/master/fakerclass.html#examples) in the documentation).

Here's the [list of all available locales](https://faker.readthedocs.io/en/master/locales.html).

In [23]:
from faker import Faker
# set a seed for the random generator
Faker.seed(0)

Generate data with locales `de_DE` and `de_AT` with weights respectively $5$ and $2$.

The distribution of locales will be:
 - `de_DE` - $71.43\%$ of the time ($5 / (5+2)$)
 - `de_AT` - $28.57\%$ of the time ($2 / (5+2)$)


In [24]:
from collections import OrderedDict
locales = OrderedDict([
    ('de_DE', 5),
    ('de_AT', 2),
])
fake = Faker(locales)
fake.seed_instance(42)
fake.locales

['de_DE', 'de_AT']

In [25]:
fake.seed_locale('de_DE', 0)
fake.seed_locale('de_AT', 0)

In [26]:
fake.profile(fields=['name', 'birthdate', 'sex', 'blood_group',
                     'mail', 'current_location'])

{'current_location': (Decimal('26.547114'), Decimal('-10.243190')),
 'blood_group': 'B-',
 'name': 'Axel Jung',
 'sex': 'M',
 'mail': 'claragollner@gmail.com',
 'birthdate': datetime.date(2003, 4, 16)}

In [27]:
from pyspark.sql.types import *
location = StructField('current_location',
                       StructType([StructField('lat', DecimalType()),
                                   StructField('lon', DecimalType())])
                      )
schema = StructType([StructField('name', StringType()),
                     StructField('birthdate', DateType()),
                     StructField('sex', StringType()),
                     StructField('blood_group', StringType()),
                     StructField('mail', StringType()),
                     location
                     ])

In [28]:
fake.profile(fields=['name', 'birthdate', 'sex', 'blood_group',
                     'mail', 'current_location'])

{'current_location': (Decimal('79.153888'), Decimal('-0.003034')),
 'blood_group': 'B-',
 'name': 'Dr. Anita Suppan',
 'sex': 'F',
 'mail': 'schauerbenedict@kabsi.at',
 'birthdate': datetime.date(1979, 12, 28)}

In [29]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Faker demo - part 2") \
    .getOrCreate()

Create dataframe with $5\cdot10^3$ rows.

In [30]:
%%time
n = 5*10**3
d = (fake.profile(fields=['name', 'birthdate', 'sex', 'blood_group',
                          'mail', 'current_location'])
     for i in range(n))
df = spark.createDataFrame(d, schema)

CPU times: user 3.28 s, sys: 16 ms, total: 3.3 s
Wall time: 3.64 s


In [31]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- birthdate: date (nullable = true)
 |-- sex: string (nullable = true)
 |-- blood_group: string (nullable = true)
 |-- mail: string (nullable = true)
 |-- current_location: struct (nullable = true)
 |    |-- lat: decimal(10,0) (nullable = true)
 |    |-- lon: decimal(10,0) (nullable = true)



Note how `location` represents a _tuple_ data structure (a `StructType` of `StructField`s).

In [32]:
df.show(n=10, truncate=False)

+-----------------------+----------+---+-----------+-----------------------------+----------------+
|name                   |birthdate |sex|blood_group|mail                         |current_location|
+-----------------------+----------+---+-----------+-----------------------------+----------------+
|Prof. Valentine Noack  |1979-01-30|F  |B-         |maricagotthard@aol.de        |{74, 164}       |
|Magrit Graf            |1942-12-03|F  |A-         |hartungclaudio@web.de        |{-86, -34}      |
|Harriet Weller-Lindau  |1959-10-12|F  |AB+        |heserhilma@gmail.com         |{20, 126}       |
|Ing. Walfried Roskoth  |1912-02-20|M  |B-         |kambsliane@hotmail.de        |{73, 169}       |
|Alexa Loidl-Schönberger|1933-11-17|F  |O-         |hannafroehlich@gmail.com     |{-23, -117}     |
|Hans-Erich Hartmann    |1971-05-20|M  |O-         |vadimkostolzin@gmx.de        |{39, -118}      |
|Ing. Sofia Fritsch B.A.|1966-10-18|F  |A-         |weinhagehans-christian@gmx.de|{-11, 73}       |


# Save to Parquet

[Write to parquet](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=write#pyspark.sql.DataFrameWriter.parquet) file ([Parquet](http://parquet.apache.org/) is a compressed, efficient columnar data representation compatible with all frameworks in the Hadoop ecosystem):

In [33]:
df.write.mode("overwrite").parquet("fakedata.parquet")

Check the size of parquet file (it is actually a directory containing the partitions):

In [34]:
!du -h fakedata.parquet

188K	fakedata.parquet


In [35]:
!ls -lh fakedata.parquet

total 172K
-rw-r--r-- 1 root root 71K Jan  1 17:24 part-00000-ae851943-80ab-488f-8e84-70a7329fb87f-c000.snappy.parquet
-rw-r--r-- 1 root root 98K Jan  1 17:24 part-00001-ae851943-80ab-488f-8e84-70a7329fb87f-c000.snappy.parquet
-rw-r--r-- 1 root root   0 Jan  1 17:24 _SUCCESS


# Stop Spark session

Don't forget to close the Spark session when you're done!

In [36]:
spark.stop()